# MNIST using NDs

Using nested dichotomies to get a score for the MNIST dataset

Will use KNN, SVM and the combination of multiple together

In [6]:
# imports
import itertools
import numpy as np
import pandas as pd
import concurrent.futures
import numpy as np
import math
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_curve, ConfusionMatrixDisplay, auc, roc_auc_score, f1_score, confusion_matrix
from sklearn import datasets
from statistics import mean
# import includes.model as mod
import pandas as pd
from joblib import dump, load
# from includes.config import Config;
# import includes.model_functions as mf
import time
from itertools import combinations
import random
# from graphviz import Digraph
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from datetime import datetime 
import os
import argparse
from itertools import count
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from tensorflow.keras.datasets import mnist

In [ ]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text="Hello world!", speaker_wav="my/cloning/audio.wav", language="en")
# Text to speech to a file
tts.tts_to_file(text="Hello world!", speaker_wav="my/cloning/audio.wav", language="en", file_path="output.wav")

In [ ]:
# Prepare data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten the 28x28 images into a single 784-length vector per image
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Convert to a Pandas DataFrame
X_train_df = pd.DataFrame(X_train_flattened)
X_test_df = pd.DataFrame(X_test_flattened)

# Optionally, add column names (e.g., "pixel_0", "pixel_1", ..., "pixel_783")
X_train_df.columns = [f"pixel_{i}" for i in range(X_train_flattened.shape[1])]
X_test_df.columns = [f"pixel_{i}" for i in range(X_test_flattened.shape[1])]

# Add the labels as a separate column if desired
X_train_df['Y'] = y_train
X_test_df['Y'] = y_test

print(X_train_df.head())
X_train = X_train_df.sample(5000)
X_test = X_test_df

start = time.perf_counter()
dataset = filename = 'MNIST'
model_types = ['randomForest', 'LogisticRegression','xgboost','KNN','SVM']
model_types = ['knnhyper']
config = Config(dataset)

score_type = 'accuracy'
categories = tuple(X_train['Y'].unique())

In [ ]:
config.log.info('Beginning of stepwise tree finder.')
best_tree = mf.stepwise_tree_finder(config, categories, X_train, [], {}, model_types=model_types, score_type=score_type)
config.log.info('Finished stepwise tree finder.')
config.log.info(f"Took: {round(time.perf_counter()-start,3)} to do find best tree.")
model_strucs = list(best_tree.keys())
tree_types = list(best_tree.values())
config.log.info(model_strucs)
config.log.info(tree_types)
best_trained_model = mf.build_best_tree(config, X_test, X_train, y_test, score_type, tree_types, model_strucs, categories, transform_label=None)[0]
mf.graph_model(config, best_trained_model, filename, transform_label=None, model_types=model_types)

# Run Current models

We already have scores from papers, but still good to do it ourselves

In [12]:
from sklearn.multiclass import OneVsOneClassifier
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten the 28x28 images into a single 784-length vector per image
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Convert to a Pandas DataFrame
X_train_df = pd.DataFrame(X_train_flattened)
X_test_df = pd.DataFrame(X_test_flattened)

# Optionally, add column names (e.g., "pixel_0", "pixel_1", ..., "pixel_783")
X_train_df.columns = [f"pixel_{i}" for i in range(X_train_flattened.shape[1])]
X_test_df.columns = [f"pixel_{i}" for i in range(X_test_flattened.shape[1])]

# Add the labels as a separate column if desired
# X_train_df['Y'] = y_train
# X_test_df['Y'] = y_test
X_train = X_train_df
X_test = X_test_df
y_test = y_test

In [13]:
# Run with xgboost OVO
model = xgb.XGBClassifier(n_jobs = -1, objective="binary:logistic")
# model = OneVsOneClassifier(model)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.10f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.9795000000
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.98      0.98      0.98       892
           6       0.98      0.98      0.98       958
           7       0.98      0.97      0.98      1028
           8       0.98      0.98      0.98       974
           9       0.97      0.97      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

